In [1]:
# 필요한 라이브러리 가져오기
import pandas as pd
import numpy as np
import torch
import time
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import weight_norm
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam
from math import sqrt
import tensorflow as tf
from torch.utils.data import DataLoader
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, AveragePooling1D, Dropout, GRU, Dense, concatenate, TimeDistributed,  MaxPooling1D
from tensorflow.keras import backend as K
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
import optuna

In [2]:
np.random.seed(42)
torch.manual_seed(42)

# GPU 메모리 관리 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가를 필요에 따라 할당
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 특정 크기의 메모리만 할당
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        print(e)

2024-03-11 20:51:29.055081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 20:51:29.070652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 20:51:29.070744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# 평가 지표 함수
def nrmse(predictions, targets):
    rmse = sqrt(mean_squared_error(predictions, targets))
    return rmse / np.mean(targets)
def nmae(predictions, targets):
    mae_value = mean_absolute_error(predictions, targets)
    mean_target = np.mean(targets)
    nmae_value = mae_value / mean_target
    return nmae_value
def r2(predictions, targets):
    r2_value = r2_score(targets, predictions)
    return r2_value

In [4]:
file_incheon_path = '/home/aibig25/hong_sj/solar/minmax/Gwangju_v3.csv'

In [5]:
# train과 test DataFrame을 병합
data = pd.read_csv(file_incheon_path)
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Hour'] = data['Date'].dt.hour
data_copy = data.drop(['PVGeneration', 'Date'], axis = 1)
pv_value = data[['PVGeneration']]

In [6]:
test_origin_x  = data_copy[-4015:]
test_origin_y  = pv_value[-4015:] 

test_x  = data_copy[-4015+11:]
test_y  = pv_value[-4015+11:] 

train_origin_x = data_copy[:len(data_copy)-4015]
train_origin_y = pv_value[:len(data_copy)-4015]

train_x = data_copy[11:len(data_copy)-4015]
train_y = pv_value[11:len(data_copy)-4015]

In [7]:
print("Test data 개수 = {}".format(test_x.shape))
print("Test data 개수 = {}".format(test_y.shape))
print("Train data 개수 = {}".format(train_x.shape))
print("Train data 개수 = {}".format(train_y.shape))
print("Test  data 개수 = {}".format(data_copy.shape[0]))

Test data 개수 = (4004, 13)
Test data 개수 = (4004, 1)
Train data 개수 = (8019, 13)
Train data 개수 = (8019, 1)
Test  data 개수 = 12045


In [8]:
def train_batch(train_origin_x, train_origin_y, train_x):
    x_batch_tmp = []
    y_batch_tmp = []

    start = 11 # 시작점 설정
    seq = 11 # 시퀀스 길이

    for i in range(train_x.shape[0]):
        x_history = train_origin_x.iloc[start-seq:start, :].values 
        y_history = train_origin_y.iloc[start-seq:start].values 

        x_batch_tmp.append(x_history)
        y_batch_tmp.append(y_history)

        start += 1

    x_batch = np.array(x_batch_tmp)
    y_batch = np.array(y_batch_tmp)
    print(x_batch.shape)
    print(y_batch.shape)
    
    return x_batch, y_batch

# test_batch 함수도 동일한 방식으로 수정
def test_batch(test_origin_x, test_origin_y, test_x):
    x_batch_tmp = []
    y_batch_tmp = []

    start = 11 # 시작점 설정
    seq = 11 # 시퀀스 길이

    for i in range(test_x.shape[0]):
        x_history = test_origin_x.iloc[start-seq:start, :].values 
        y_history = test_origin_y.iloc[start-seq:start].values 

        x_batch_tmp.append(x_history)
        y_batch_tmp.append(y_history)

        start += 1

    x_batch = np.array(x_batch_tmp)
    y_batch = np.array(y_batch_tmp)
    print(x_batch.shape)
    print(y_batch.shape)   
    
    return x_batch, y_batch

x_batch, y_batch = train_batch(train_origin_x, train_origin_y, train_x)
x_test_batch, y_test_batch = test_batch(test_origin_x, test_origin_y, test_x)

(8019, 11, 13)
(8019, 11, 1)
(4004, 11, 13)
(4004, 11, 1)


In [9]:
class GRU_CNN_Model(tf.keras.Model):
    def __init__(self, num_features, num_filters, kernel_size, gru_hidden_size, dense_size, dropout_rate):
        super(GRU_CNN_Model, self).__init__()
        # GRU layer
        self.gru = GRU(units=gru_hidden_size, return_sequences=True)
        self.dropout1 = Dropout(rate=dropout_rate)
        # Conv1D layer
        self.conv1 = Conv1D(filters=num_filters, kernel_size=kernel_size, padding='same', activation='relu')
        # Removed MaxPooling1D layer to maintain sequence length
        # TimeDistributed Dense layer for sequence output
        self.time_dist_dense1 = TimeDistributed(Dense(units=dense_size, activation='relu'))
        self.dropout2 = Dropout(rate=dropout_rate)
        self.time_dist_dense2 = TimeDistributed(Dense(units=1, activation='linear'))

    def call(self, inputs):
        # GRU layer
        x = self.gru(inputs)
        x = self.dropout1(x)
        # Conv1D layer
        x = self.conv1(x)
        # Applying TimeDistributed Dense without reducing sequence length
        x = self.time_dist_dense1(x)
        x = self.dropout2(x)
        x = self.time_dist_dense2(x)
        return x
        
# Define the input shape as given
sequence_length = 11  # As per the input specification
num_features = x_batch.shape[2]  # Placeholder, replace x_batch.shape[2] with actual value

# Model instantiation
inputs = Input(shape=(sequence_length, num_features))
model = GRU_CNN_Model(num_features=num_features, num_filters=64, kernel_size=3, gru_hidden_size=32, dense_size=128, dropout_rate=0.5)
outputs = model(inputs)

# Compile the model (Example)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

# Summary to verify the output shape
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11, 13)]          0         
                                                                 
 gru_cnn__model (GRU_CNN_Mod  (None, 11, 1)            19169     
 el)                                                             
                                                                 
Total params: 19,169
Trainable params: 19,169
Non-trainable params: 0
_________________________________________________________________


2024-03-11 20:51:30.161135: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 20:51:30.161897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 20:51:30.162008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 20:51:30.162052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [10]:
def create_model(trial):
    num_features = 13  # 예시 데이터셋의 특성 수, 실제 데이터에 맞게 조정하세요
    gru_hidden_size = trial.suggest_categorical('gru_hidden_size', [16, 32, 64])
    num_filters = trial.suggest_categorical('num_filters', [16, 32, 64])
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5, 7])
    dense_size = trial.suggest_categorical('dense_size', [16, 32, 64])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    
    model = GRU_CNN_Model(num_features=num_features, num_filters=num_filters, kernel_size=kernel_size,
                          gru_hidden_size=gru_hidden_size, dense_size=dense_size, dropout_rate=dropout_rate)
    optimizer = Adam(lr=1e-3)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

def objective(trial):
    model = create_model(trial)

    # 모델 학습
    model.fit(x_batch, y_batch, epochs = 100, batch_size=11, verbose=0)
    
    # 평가
    loss, nrmse = model.evaluate(x_test_batch, y_test_batch, verbose=0)
    return nrmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-03-11 20:51:30,884] A new study created in memory with name: no-name-5058ecd1-5e81-426e-baaa-663d2e08a9e2
/tmp/ipykernel_295018/1467116652.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/home/aibig25/anaconda3/envs/hong_sj/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2024-03-11 20:51:31.706100: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2024-03-11 20:51:32.175895: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-03-11 20:51:32.175936: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-0

Number of finished trials: 100
Best trial: {'gru_hidden_size': 64, 'num_filters': 64, 'kernel_size': 7, 'dense_size': 16, 'dropout_rate': 0.27359237767612987}


In [11]:
best_params = study.best_trial.params
model = create_model(study.best_trial)  # 최적의 하이퍼파라미터를 사용하여 모델 생성

# 최적의 하이퍼파라미터로 모델 학습
model.fit(x_batch, y_batch, epochs = 100, batch_size = 11)

# 최종 모델 평가
loss, mse = model.evaluate(x_test_batch, y_test_batch)
print(f'Test MAE: {mse}')

/tmp/ipykernel_295018/1467116652.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)


Epoch 1/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0896 - mae: 0.2566
Epoch 2/100
729/729 [==============================] - 4s 6ms/step - loss: 0.0799 - mae: 0.2497
Epoch 3/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0794 - mae: 0.2497
Epoch 4/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0793 - mae: 0.2497
Epoch 5/100
729/729 [==============================] - 5s 7ms/step - loss: 0.0793 - mae: 0.2497
Epoch 6/100
729/729 [==============================] - 5s 7ms/step - loss: 0.0793 - mae: 0.2497
Epoch 7/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0793 - mae: 0.2497
Epoch 8/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0793 - mae: 0.2498
Epoch 9/100
729/729 [==============================] - 5s 6ms/step - loss: 0.0793 - mae: 0.2496
Epoch 10/100
729/729 [==============================] - 5s 7ms/step - loss: 0.0793 - mae: 0.2497
Epoch 11/100
729/729 [=================

In [12]:
# 성능 평가
mae_pred = model.predict(x_test_batch)
mae_pred_re = mae_pred.reshape(x_test_batch.shape[0], -1)
y_test_batch_re = y_test_batch.reshape(y_test_batch.shape[0], -1)

print(mae_pred_re.shape)
print(y_test_batch_re.shape)

(4004, 11)
(4004, 11)


In [13]:
mae_result = mean_absolute_error(mae_pred_re, y_test_batch_re)
mse_result = mean_squared_error(mae_pred_re, y_test_batch_re)
rmse_result = (mse_result)**0.5
nrmse_result = nrmse(mae_pred_re, y_test_batch_re)
nmae_result = nmae(mae_pred_re, y_test_batch_re)
r2_result = r2(mae_pred_re, y_test_batch_re)

print("MAE = {}".format((mae_result)))
print("RMSE = {}".format((rmse_result)))
print("NRMSE = {}".format((nrmse_result)))
print("NMAE = {}".format(nmae_result))
print("R2 = {}".format(r2_result))

MAE = 0.23848577304663968
RMSE = 0.27156171067732326
NRMSE = 0.7739166423355698
NMAE = 0.6796543896439317
R2 = -0.0028794254908672823


In [14]:
def mae_11_points(pred, y):
    mae_11_points = []

    for i in range(pred.shape[1]):
        mae_point = np.mean(np.abs(pred[:,i] - y[:, i]))
        mae_11_points.append(mae_point)

    return mae_11_points

def rmse_11_points(pred, y):
    rmse_11 = []
    
    for i in range(pred.shape[1]):
        rmse_point = np.sqrt(np.mean(np.square(pred[:,i]-y[:,i])))
        rmse_11.append(rmse_point)
        
    return rmse_11
    
def nrmse_11_points(pred, y):
    nrmse_11_points = []
    for i in range(pred.shape[1]):
        rmse_point = np.sqrt(np.mean((pred[:,i] - y[:,i]) ** 2))
        nrmse_point = rmse_point / np.mean(y[:,i])
        nrmse_11_points.append(nrmse_point)
    return nrmse_11_points

def nmae_11_points(pred, y):
    nmae_11_points = []
    for i in range(pred.shape[1]):
        mae_point = np.mean(np.abs(pred[:,i] - y[:,i]))
        mean_target = np.mean(y[:,i])
        nmae_point = mae_point / mean_target
        nmae_11_points.append(nmae_point)
    return nmae_11_points

def r2_11_points(pred, y):
    r2_11_points = []
    for i in range(pred.shape[1]):
        pred_column = pred[:,i].reshape(-1, 1)
        y_column = y[:,i].reshape(-1, 1)
        r2_point = r2_score(y_column, pred_column)
        r2_11_points.append(r2_point)
    return r2_11_points

In [15]:
mae_11 = np.round(mae_11_points(mae_pred_re, y_test_batch_re), 6)
rmse_11 = np.round(rmse_11_points(mae_pred_re, y_test_batch_re), 6)
nrmse_11 = np.round(nrmse_11_points(mae_pred_re, y_test_batch_re), 6)
nmae_11 = np.round(nmae_11_points(mae_pred_re, y_test_batch_re), 6)
r2_11 = np.round(r2_11_points(mae_pred_re, y_test_batch_re), 6)

print("MAE = {}".format((mae_11)))
print("RMSE = {}".format(rmse_11))
print("NRMSE = {}".format(nrmse_11))
print("NMAE = {}".format(nmae_11))
print("R2 = {}".format(r2_11))

MAE = [0.238402 0.238402 0.23839  0.238373 0.238401 0.238467 0.238561 0.238602
 0.238618 0.238573 0.238554]
RMSE = [0.271491 0.271491 0.271479 0.271476 0.271494 0.271549 0.271618 0.271652
 0.27166  0.271643 0.271626]
NRMSE = [0.774145 0.774145 0.774086 0.774021 0.774011 0.774021 0.774009 0.773852
 0.773735 0.773574 0.773486]
NMAE = [0.679792 0.679792 0.679737 0.679638 0.679664 0.679725 0.679811 0.679702
 0.679628 0.679399 0.679311]
R2 = [-0.002958 -0.002958 -0.002954 -0.002944 -0.002932 -0.002904 -0.002866
 -0.002819 -0.002795 -0.002775 -0.002768]
